In [16]:
import requests
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
import json

In [13]:
load_dotenv()

True

In [14]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [15]:
eval("2+2")

4

In [17]:
with open('test_data.json', "r") as file:
    test_data = json.load(file)

In [18]:
def is_complex_question(question):
    """
    Identyfikuje, czy pytanie jest złożone i wymaga LLM.
    """
    try:
        eval(question)  # Jeśli pytanie można obliczyć programowo, nie jest złożone
        return False
    except:
        return True  # Pytanie wymaga LLM


In [23]:
len(test_data['test-data'])

2137

In [24]:
test_data['test-data'][0]

{'question': '84 + 41', 'answer': 125}

In [25]:
simple_tasks = []
complex_tasks = []

for q_a in test_data['test-data']:
    q = q_a['question']
    try:
        eval(q)
        simple_tasks.append(q)
    except:
        complex_tasks.append(q)

In [19]:
simple_tasks = []
complex_tasks = []

for item in test_data.get("test-data", []):
    question = item.get("question", "")
    if is_complex_question(question):
        complex_tasks.append(item)
    else:
        simple_tasks.append(item)

In [ ]:
client = OpenAI()

#api shooting
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Jesteś asystentem. Pamiętaj, że: \
                                    - stolicą Polski jest Kraków \
                                    - znana liczba z książki Autostopem przez Galaktykę to 69 \
                                    - Aktualny rok to 1999 \
                                    Dla innych pytań odpowiadasz w pełni prawdziwie."},
        {"role": "user", "content": f"Odpowiedz tylko samą liczbą na podane pytanie: {response.json()['text']} "}
    ]
)
captcha_answer = completion.choices[0].message.content

In [36]:
# Funkcja do rozpoznania, czy pytanie jest złożone
def is_simple_question(question_text):
    try:
        eval(question_text)  # Próba interpretacji matematycznej
        return True
    except:
        return False

# Funkcja do przetwarzania prostych pytań
def process_simple_question(question_text):
    try:
        return eval(question_text)
    except Exception as e:
        print(f"Błąd w przetwarzaniu pytania '{question_text}': {e}")
        return None

# Funkcja do przetwarzania pytań złożonych za pomocą LLM
def process_complex_question_with_llm(question_text):
    client = OpenAI()
    try:
        completion =  client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Jesteś asystentem."},
                {"role": "user", "content": question_text}
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Błąd API: {e}")
        return None

# Funkcja do przetwarzania jednego rekordu
def process_record(record):
    # Obsługa pola "question"
    if "question" in record:
        question = record["question"]
        correct_answer = process_simple_question(question) if is_simple_question(question) else None
        if correct_answer is not None and record.get("answer") != correct_answer:
            print(f"Poprawiam 'answer' dla '{question}': było {record['answer']}, poprawne {correct_answer}")
            record["answer"] = correct_answer

    # Obsługa pola "test"
    if "test" in record:
        test = record["test"]
        question_test = test.get("q")
        if question_test:
            print(f"Przetwarzam test: {question_test}")
            test_answer = process_complex_question_with_llm(question_test)
            test["a"] = test_answer
            print(f"Odpowiedź na test '{question_test}': {test_answer}")
    
    return record

# Funkcja główna do przetwarzania listy rekordów
def process_data(data):
    processed_data = []
    for record in data:
        processed_data.append(process_record(record))
    return processed_data

In [37]:
print("Rozpoczynam przetwarzanie danych...")
processed_data = process_data(test_data['test-data'])

Rozpoczynam przetwarzanie danych...
Przetwarzam test: What is the capital city of Germany?
Odpowiedź na test 'What is the capital city of Germany?': The capital city of Germany is Berlin.
Przetwarzam test: What is the capital city of France?
Odpowiedź na test 'What is the capital city of France?': The capital city of France is Paris.
Przetwarzam test: What is the capital city of Poland?
Odpowiedź na test 'What is the capital city of Poland?': The capital city of Poland is Warsaw.
Przetwarzam test: name of the 2020 USA president
Odpowiedź na test 'name of the 2020 USA president': The president of the United States in 2020 was Donald Trump.


In [39]:
test_data

{'apikey': '%PUT-YOUR-API-KEY-HERE%',
 'description': 'This is simple calibration data used for testing purposes. Do not use it in production environment!',
 'copyright': 'Copyright (C) 2238 by BanAN Technologies Inc.',
 'test-data': [{'question': '84 + 41', 'answer': 125},
  {'question': '54 + 38', 'answer': 92},
  {'question': '12 + 49', 'answer': 61},
  {'question': '35 + 42', 'answer': 77},
  {'question': '100 + 83', 'answer': 183},
  {'question': '79 + 43', 'answer': 122},
  {'question': '1 + 97', 'answer': 98},
  {'question': '75 + 29', 'answer': 104},
  {'question': '89 + 11', 'answer': 100},
  {'question': '10 + 45', 'answer': 55},
  {'question': '37 + 70', 'answer': 107},
  {'question': '81 + 29', 'answer': 110},
  {'question': '23 + 2', 'answer': 25},
  {'question': '42 + 41', 'answer': 83},
  {'question': '25 + 85', 'answer': 110},
  {'question': '77 + 24', 'answer': 101},
  {'question': '64 + 1', 'answer': 65},
  {'question': '82 + 22', 'answer': 104},
  {'question': '44 + 

In [69]:
response_poligon = requests.get("https://poligon.aidevs.pl/dane.txt")

In [93]:
output_json = {
    'task': 'JSON',
    'apikey': os.getenv('AI_DEVS_3_API_KEY'),
    'answer': ready_test_data
}

In [94]:
response = requests.post("https://centrala.ag3nts.org/report", json=output_json)

In [95]:
response.json()

{'code': 0, 'message': '{{FLG:JSONFIX}}'}

In [47]:
processed_data[0]

{'question': '84 + 41', 'answer': 125}

In [48]:
processed_data_tests = [test for test in processed_data if 'test' in test.keys()]

In [49]:
len(processed_data_tests)

4

In [50]:
processed_data_tests

[{'question': '82 + 9',
  'answer': 91,
  'test': {'q': 'What is the capital city of Germany?',
   'a': 'The capital city of Germany is Berlin.'}},
 {'question': '18 + 64',
  'answer': 82,
  'test': {'q': 'What is the capital city of France?',
   'a': 'The capital city of France is Paris.'}},
 {'question': '67 + 50',
  'answer': 117,
  'test': {'q': 'What is the capital city of Poland?',
   'a': 'The capital city of Poland is Warsaw.'}},
 {'question': '47 + 18',
  'answer': 65,
  'test': {'q': 'name of the 2020 USA president',
   'a': 'The president of the United States in 2020 was Donald Trump.'}}]

In [82]:
ready_test_data = test_data

In [83]:
ready_test_data['apikey'] = os.getenv('AI_DEVS_3_API_KEY')
ready_test_data['test-data'] = processed_data